In [ ]:
exec(open("init_notebook.py").read())

In [ ]:
import pyodrx, extensions
from library.Configuration import Configuration
import numpy as np
from junctions.RoadBuilder import RoadBuilder
from junctions.JunctionBuilder import JunctionBuilder

In [ ]:
roadBuilder = RoadBuilder()
junctionBuilder = JunctionBuilder()

In [ ]:
roads = []
roads.append(pyodrx.create_straight_road(0, 10))
roads.append(roadBuilder.createSimpleCurve(1, np.pi/1.5, True, curvature = 0.9))
roads.append(pyodrx.create_straight_road(2, 10))

roads[0].add_successor(pyodrx.ElementType.road,1,pyodrx.ContactPoint.end)

roads[1].add_predecessor(pyodrx.ElementType.road,0,pyodrx.ContactPoint.start)
roads[1].add_successor(pyodrx.ElementType.road,2,pyodrx.ContactPoint.end)

roads[2].add_predecessor(pyodrx.ElementType.road,1,pyodrx.ContactPoint.start)

junction = junctionBuilder.createJunctionForASeriesOfRoads(roads)



In [ ]:
odrName = "curve_test"
odr = extensions.createOdr(odrName, roads, [])
odr.adjust_roads_and_lanes()
